In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
import pandas as pd

jar_files = [
    "/util/clickhouse-jdbc-0.3.2-all.jar"
]

# Initialize Spark session with JARs
spark = SparkSession.builder \
    .appName("combina_shapes") \
    .master("spark://spark-master:7077") \
    .config("spark.jars", ",".join(jar_files)) \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.extensions","io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.hadoop.fs.s3a.access.key","admin") \
    .config("spark.hadoop.fs.s3a.secret.key","minioadmin") \
    .config("spark.hadoop.fs.s3a.endpoint","http://minio:9000") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()


url = "jdbc:ch://clickhouse:8123/cameraaberta"
user = "admin" 
password = "admin"  
driver = "com.clickhouse.jdbc.ClickHouseDriver"

display(spark)

In [2]:
# df_previsao = pd.read_json(f'./previsao.json', ignore)
# print(df_previsao.shape)
# df_previsao.head()

In [ ]:
df_stops = spark.read.option("inferSchema", True).option("header", "true") \
    .csv(f"s3a://raw/stops.txt").toPandas()
# show data
df_stops.head(10)

In [ ]:
df_trips = spark.read.option("inferSchema", True).option("header", "true") \
    .csv(f"s3a://raw/trips.txt").toPandas()
# show data
df_trips.head(10)

In [ ]:
df_shapes = pd.read_csv('trips_stops.csv')
# show data
df_shapes.head(10)

In [ ]:
random.randint(-2, +10)

In [ ]:
import random


list_trips = df_trips['trip_id'].values

data = []

for s in df_shapes.values:
    data.append({"vs_ta" : 1, "trip_id":random.choice(list_trips),  'stop_id': float(s[6]), 'vs_p': '1', 'shape_id': int(s[1]), 'vs_t': random.randint(-2, +10) })
    
data[:10]
    

In [ ]:
df_chegadas = spark.createDataFrame(data)
print('gravando Clickhouse')
df_chegadas.write \
    .format("jdbc") \
    .option("driver", driver) \
    .option("url", url) \
    .option("user", user) \
    .option("password", password) \
    .option("dbtable", "chegadas") \
    .option("createTableOptions", "ENGINE=Log()") \
    .mode("overwrite") \
    .save()
print('gravando Minio')
df_chegadas.write.csv(f"s3a://trusted/chegadas.csv")